# Copying SuperMAG to S3

Copies locally stored files to S3, and logs into a database for tracking purposes.

* Reads MANIFEST.csv to get filenames and any metadata
    * filenames can include subdirectories. Are presumed to be in local directory LOCALDIR
* Writes final files to location defined by 'S3DIR' and updates DB fields "s3key" to point to new location, "metadata" to contain all metadata (for later parsing)

Database is sqlite3, named 'Supermag_log.db', contains fields 'filename' (original Supermag filename), 's3key', 'status' (0=not yet copied over, 1 = successfully copied over), 'metadata' (direct copy of original MANIFEST.csv line in its entirely, for later processing into the index)

TEST = True case copies 10 files to TEST_S3DIR

INGEST = True reads the MANIFEST.csv into the database (is safe, won't overwrite processed files).

In [4]:
import sqlite3
import re
import time
import itertools
import boto3
import os.path
from botocore.exceptions import ClientError
from multiprocessing.pool import ThreadPool
import multiprocessing
#import multiprocessing_logging
import warnings

dbname = "Supermag_log.db"
LOCALDIR = "."
S3DIR = "gov-nasa-hdrl-data1/contributed/supermag/"
INGEST = True # initial population of database from manifest
TEST = True
if TEST:
    S3DIR = "helio-public/skantunes/supermag-test/"

In [27]:
def ingest_filelist(manifest='MANIFEST.csv'):
    # supermag-defined way of getting list of files to copy and register, likely MANIFEST.csv
    # format  filename, multiple_optional_metadata
    with open(manifest,'r') as fin:
        for line in fin:
            filename = line.split(',')[0]
            insertDB_meta(filename,meta)
        conn.commit()

In [6]:
def testcred(input):
    import cred
    credentials=cred.altcred()
    print("cred test: ",input,credentials)
    retval = (credentials.access_key,"test")
    return retval

In [7]:
def generate_s3key(s3dir, filename):
    bucket = s3dir.split('/')[0]
    oname = '/'.join(s3dir.split('/')[1:]) + filename
    s3key = "s3://" + s3dir + filename
    return s3key, bucket, oname

In [8]:
def dbget_filelist(status=0):
    state_fetch = f"SELECT filename from supermag where status={status}"
    rows=cursor.execute(state_fetch).fetchall()
    return rows

In [9]:
def lookup_db(dbname, key, keyval):
    state_fetch = f"SELECT * from supermag where {key}='{keyval}'"
    rows=cursor.execute(state_fetch).fetchall()
    return rows

In [26]:
def insertDB_meta(filename, meta):
    state_change = f"INSERT OR IGNORE INTO supermag (filename,metadata,status) VALUES ('{filename}','{meta}',0);"
    cursor.execute(state_change)

In [11]:
def updateDB_s3key(filename, s3key):
    state_change = f"UPDATE supermag set s3key='{s3key}', status = 1 WHERE filename='{filename}'"
    cursor.execute(state_change)

In [12]:
def countdb():
    query = "SELECT COUNT(s3key) from supermag"
    rows = cursor.execute(query).fetchall()
    print("Number of s3keys (files): ",rows[0][0])
    return rows[0][0]

Number of s3keys (files):  0


0

In [13]:
def copy_local_to_s3(filename, s3dir=S3DIR, localdir=LOCALDIR, profile=None):
    s3key, bucket, oname = generate_s3key(s3dir, filename)
    print(f"bucket:{bucket}, oname:{oname}, full s3key: {s3key}")
    status = False
    #s3_client = boto3.client('s3',aws_access_key_id=credentials.access_key,aws_secret_access_key=credentials.secret_key,aws_session_token=credentials.token)
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(LOCALDIR+'/'+filename, bucket, oname)
        return (filename, s3key)
    except ClientError as e:
        print("Failed to copy to S3:",s3key,e)
        return None
    except:
        print("Failed to copy to S3:",s3key)
        return None

In [28]:
conn = sqlite3.connect(dbname) #, isolation_level=None)
cursor = conn.cursor()
create = "CREATE TABLE IF NOT EXISTS supermag (filename TEXT PRIMARY KEY, s3key text, metadata TEXT, status INT);"
cursor.execute(create)
if INGEST: ingest_filelist()

FileNotFoundError: [Errno 2] No such file or directory: 'MANIFEST.csv'

In [17]:
if TEST:
    filename="test_anim.gif"
    pair = copy_local_to_s3(filename)
    if pair != None: updateDB(pair[0], pair[1])
    countdb()

bucket:helio-public, oname:skantunes/supermag-test/test_anim.gif, full s3key: s3://helio-public/skantunes/supermag-test/test_anim.gif
Failed to copy to S3: s3://helio-public/skantunes/supermag-test/test_anim.gif
Number of s3keys (files):  0


In [19]:
#PRODUCTION
#import logging
myfiles = dbget_filelist()
if TEST: myfiles = myfiles[0:10]

now=time.time()
startingcount = countdb()
finalcount = 0
popsize=len(myfiles)

print("Number of CPU cores available: ",multiprocessing.cpu_count())
num_threads = multiprocessing.cpu_count()        
pool = multiprocessing.Pool(num_threads)
return_vals = pool.map(copy_local_to_s3, myfiles)
for pair in return_vals:
    if pair != None:
        updateDB(pair[0], pair[1])
        finalcount += 1
conn.commit()

endingcount = countdb()
rate = (time.time()-now)/popsize
print(f"Time to process {popsize} files was {(time.time()-now)/60} minutes, {rate} sec/file, added {endingcount-startingcount} new files to S3")
print(f"There are now {countdb()} files registered.")

Number of s3keys (files):  0
Number of CPU cores available:  16
Number of s3keys (files):  0


ZeroDivisionError: float division by zero

In [ ]:
conn.commit()
conn.close()